# Árboles de decisión y EDA
La clase pasada estuvimos viendo un poco de estadística y EDA (exploratory data analysis).

La primer mitad vamos a trabajar con el dataset de flores iris y la segunda mitad es para que ustedes trabajen con el dataset breast-cancer-wisconsin.data

## Iris

Iris es un dataset clásico para aquellos incursionando en el campo de Machine Learning. Se trata de una colección de 150 flores que deben ser categorizadas en 3 clases (Setosa, Versicolor, Virginica). Esto debe realizarse a partir de medidas tomadas sobre cada flor.


Abajo comenzamos explorando el dataset y dejamos algunas tareas para que ustedes completen a medida que van avanzando. 



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

iris_dataset = load_iris()
iris_dataset.keys()


In [ ]:
iris_dataset['feature_names']

### Explorar el dataset

Lo primero que vamos a tener que hacer es cargar el dataset en un dataframe

In [ ]:
#cargamos data al dataframe
df = pd.DataFrame(iris_dataset['data'], columns=iris_dataset['feature_names'])
#le agregamos el target
df['target'] = iris_dataset['target']

In [ ]:
df

In [ ]:
df.describe()

### Plotting some features

In [ ]:
colors = np.array(["r", "g", "b"])
plt.scatter(iris_dataset.data[:, 0], iris_dataset.data[:, 1], color=colors[iris_dataset.target])
plt.title("Sepal Length and Width")
plt.xlabel("Sepal Length")
plt.ylabel("Sepal Width")
plt.show()

### Ejercicio 1
Grafique el largo y ancho de los pétalos 

In [ ]:
# Su código

¿Qué puede observar en ambas gráficas respecto a las tres clases? ¿Qué conclusiones puede tomar?


# Clasificación

Para este problema de ejemplo vamos a entrenar un modelo sencillo basado en [Árboles de Decisión](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

Los árboles de decisión son algoritmos de Machine Learning supervisado. Se construye un arbol binario (cada nodo tiene dos ramas). Los valores objetivos se presentan en las hojas de los árboles.

Antes de poder llegar a eso debemos dividir el dataset en conjuntos adecuados para **entrenar** y **testar** el modelo.


### Ejercicio 2

¿Por qué debemos separar el set de datos? Es decir, ¿por qué no podemos validar el resultado con los mismos datos con los que entrenamos?

## Split train-test

Usando sklearn separamos el dataset (y los targets) iniciales en dos: un set de entrenamiento y uno de test.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    df.drop('target', axis=1),
    df['target'],
    test_size=(1.0/3), random_state=42)

print(f'df shape = {df.shape}')
print('------------------------')
print(f'X_train shape = {X_train.shape}\n------------------------')
print(f'Y_train shape = {Y_train.shape}\n------------------------')
print(f'X_test shape = {X_test.shape}\n------------------------')
print(f'Y_test shape = {Y_test.shape}')

Una vez separados los datos podemos crear un modelo y entrenarlo. Para ello, necesitamos un objeto de la clase `DecisionTreeClassifier`.

Una vez creado el mismo, llamamos al método `fit` con los datos de entrenamiento para entrenarlo y luego podemos usar los métodos predict para obtener predicciones de las clases para los datos de test.


Entrenamiento de un árbol de decisión de profundidad máxima 3 para clasificar los datos

Recuerden que pueden acceder a la documentación [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

arbol = DecisionTreeClassifier(max_depth=3, criterion = 'entropy')

# Hacemos el fit
arbol.fit(X_train, Y_train)

Plotting the Decision Tree

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(15,10))
tree.plot_tree(arbol, 
                   feature_names=iris_dataset.feature_names,  
                   class_names=iris_dataset.target_names,
                   filled=True,)

In [ ]:
fig.savefig("decision_tree.png")

### Ejercicio 3

La accuracy es el porcentaje de datos que fueron clasificados correctamente

¿Cuál fue la accuracy de los datos de test?


In [ ]:
from sklearn.metrics import accuracy_score

Y_pred_test = arbol.predict(X_test)
print("Accuracy de datos de test:", accuracy_score(Y_pred_test, Y_test))

### Ejercicio 4

Armar un arbol de altura máxima 2 ¿Cuál de los dos es mejor?

In [ ]:
#arbol2 = ...


Matriz de Confusión

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
Y_pred1 = arbol.predict(X_test)
Y_pred2 = arbol2.predict(X_test)
print("Confusion Matrix Arbol 1")       
print(confusion_matrix(Y_test, Y_pred1))
print("Confusion Matrix Arbol 2")       
print(confusion_matrix(Y_test, Y_pred2))
print("--------------------------------")
print("Classification report Arbol 1")
print(classification_report(Y_test, Y_pred1, target_names = iris_dataset["target_names"]))
print(classification_report(Y_test, Y_pred2, target_names = iris_dataset["target_names"]))

También podemos imprimir la matriz de confusión más linda haciendo un plot

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(Y_test, Y_pred1)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Breast Cancer Wisconsin

El dataset de Iris es muy sencillo y muy fácilmente podemos obtener 100% de precisión con un modelo simple. La tarea siguiente implica el uso de un dataset real para el cual van a tener que entrenar un Árbol de decisión para detectar cáncer de mama a partir de distintas mediciones. 

La descripción del dataset la pueden encontrar aquí: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29

Cargamos el nuevo df

In [ ]:
# breast_cancer_df = pd...

Si hacemos `breast_cancer_df.head()` podemos ver que las columnas no tienen nombre

In [ ]:
breast_cancer_df.head()

Le podemos agregar el nombre a las columnas

In [ ]:
breast_cancer_df.columns = ["Id", "clump_thicknes", "cell_size", "cell_shape", "adhesion", "epithelial_cell_size", "nuclei", "chromatin", "nucleoli", "mitoses", "target"]
breast_cancer_df.head()

In [ ]:
# En el dataset las clases son 2 y 4 que representan Benigno y Maligno, vamos a reemplazarlas por 0 y 1 por simplicidad.
breast_cancer_df["target"].replace(2, 0, inplace=True)
breast_cancer_df["target"].replace(4, 1, inplace=True)

# Opcion A) Eliminamos la columna de ID ya que no nos interesa
breast_cancer_df.drop(columns="Id", inplace=True)

# Opcion B) Hacemos que el ID sea el Index
#breast_cancer_df.set_index("Id", inplace = True)
breast_cancer_df.head()


In [ ]:
# Finalmente, contamos cuantos ejemplos son beningnos y cuantos no (0 = B, 1 = M)
breast_cancer_df["target"].value_counts()

In [ ]:
breast_cancer_df["nuclei"].value_counts()

In [ ]:
breast_cancer_df[breast_cancer_df["nuclei"]=='?']

In [ ]:
#Quitamos observaciones que tienen '?' en Nuclei quantity
breast_cancer_df.drop(breast_cancer_df[breast_cancer_df["nuclei"]=='?'].index, inplace = True)

### Ejercicio 6

Realice una exploración del dataset como hicimos para el caso de Iris.

### Ejercicio 7

Entrene al menos 2 arboles de Decisión para este set de datos. Recuerde separar los datos en conjuntos de entrenamiento y test.

### Ejercicio 8
¿Cuál de los árboles es el mejor?
Imprima la matriz de confusión del mejor modelo